# Base notebook

We used this notebook to develop plotting and loading functions on the icosahedral grid. In the end, it wasn't used to produce results for the thesis.

In [1]:
import os
import glob

import numpy as np
import math


import gzip
import netCDF4 as nc
import pickle

import matplotlib.pyplot as plt
import matplotlib


import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point


from icosahedron import Icosahedron, rand_rotation_icosahedron, rand_rotation_matrix, plot_voronoi, plot_voronoi_charts

In [2]:
DIRECTORY_DATASETS_INTERPOLATED = "Datasets/Interpolated/"
DIRECTORY_DATASETS_ORIGINAL = "Datasets/Original/"
DIRECTORY_IMAGES = "Images/"
DIRECTORY_OUTPUTS = "Output/Compare_UNet_architectures/"

# Give criteria for the dataset and open all files that match these criteria

In [3]:
# dataset parameters. To leave unspecified use: "*"
PREFIX = "HadCM3-ico"
RESOLUTION = 5
DO_SHUFFLE = False
INTERPOLATION = "cons1"
INTERPOLATE_CORNERS = True
ALL_VARIABLES = np.sort(["temp_1","precip","dO18"]) # get all possible combinations # np.sort(["temp_1","precip","dO18","p"])
DSET_NR = "*"

# helping vars
shuffle_dict = {True:"shuffle", False:"no-shuffle", "*": "*"}
corners_dict = {True: "interp-corners", False: "zero-fill-corners", "*": "*"}

# wildcards can be used in this filename.
DATASET_FOLDER = "{}_{}_{}_{}_{}_{}_".format(PREFIX, RESOLUTION, shuffle_dict[DO_SHUFFLE], INTERPOLATION, 
                                         corners_dict[INTERPOLATE_CORNERS], DSET_NR)
DATASET_FOLDER = DATASET_FOLDER + "-".join(ALL_VARIABLES)
DATASET_FOLDER = os.path.join(DIRECTORY_OUTPUTS, DATASET_FOLDER)


dataset_description_files = []  # list to collect the paths to the configuration files
dataset_folders = []  # list to collect all the matching folders.
folder_exists = False

print("Data folders matching the given specifications:")
for dirname in glob.glob(DATASET_FOLDER):
    print(dirname)
    folder_exists = True
    dataset_folders.append(dirname)
    dataset_description_files.append(os.path.join(dirname, "dataset-description.gz"))
if folder_exists == False:
    raise OSError("There exists no folder for the given specifications")

dataset_descriptions = []
for i, file in enumerate(dataset_description_files):
    with gzip.open(file, 'rb') as f:
        dataset_descriptions.append(pickle.load(f))

Data folders matching the given specifications:
Output/Compare_UNet_architectures/HadCM3-ico_5_no-shuffle_cons1_interp-corners_1_dO18-precip-temp_1


In [4]:
def check_dict_conditions(dic, conditions):
    """
    Test whether dict "dic" fullfills the given conditions "conditions". 
    The latter are given as a array of key-value pairs.
    """
    for key, values in conditions:
        if key in dic.keys():
            if not dic[key] in values:
                return False
        else:
            return False
    return True

In [5]:
model_descriptions = []
testset_predictions = []
indices_dataset = [] # store the index of the dataset-directory to which each run belongs
model_descriptions_paths = []

# specify the conditions that we want the runs to match
conditions = []# [("NUM_EPOCHS",3)]


print("Available runs from all previously specified folders, that match specifications:")

for i, dataset_folder in enumerate(dataset_folders):
    subdirs = [d for d in os.listdir(dataset_folder) if os.path.isdir(os.path.join(dataset_folder, d))]
    for subdir in subdirs:
        files = [f for f in os.listdir(os.path.join(dataset_folder, subdir)) if os.path.isfile(os.path.join(dataset_folder, subdir, f))]
        if "model_training_description.gz" in files and "predictions.gz" in files:        
            # this is a valid description of a model run. Store path and print description
            with gzip.open(os.path.join(dataset_folder, subdir, "model_training_description.gz"), 'rb') as f:
                tmp_description = pickle.load(f)
                if check_dict_conditions(tmp_description, conditions):  # check if the description satisfies our conditions

                    model_descriptions.append(tmp_description)      
                    indices_dataset.append(i)
                    model_descriptions_paths.append(os.path.join(subdir, file))
                    print("Path: ", subdir)
                    for key, value in model_descriptions[-1].items():
                        print(key,": ", value)
                    print("\n")

                    with gzip.open(os.path.join(dataset_folder, subdir, "predictions.gz"), 'rb') as g:
                        testset_predictions.append(pickle.load(g))       
        else:
            print("Encountered Invalid directory")

Available runs from all previously specified folders, that match specifications:
Path:  -0xb4eecf0bc51a89e
MODELTYPE :  UNet
S_MODE_PREDICTORS :  ('Pixelwise', 'Pixelwise')
S_MODE_TARGETS :  ('Pixelwise',)
RUN_NR :  5
NUM_EPOCHS :  early_stopping
BATCH_SIZE :  8
LEARNING_RATE :  0.005
DEPTH :  3
IN_CHANNELS :  2
CHANNELS_FIRST_CONV :  32
OUT_CHANNELS :  1
FMAPS :  (32, 32, 64, 64)
ACTIVATION :  <class 'torch.nn.modules.activation.ReLU'>
NORMALIZATION :  <class 'torch.nn.modules.batchnorm.BatchNorm3d'>
loss :  MSELoss
optimizer :  Adam
#params :  1884369


Path:  0x73a3d33d45d81cf6
MODELTYPE :  UNet
S_MODE_PREDICTORS :  ('Pixelwise', 'Pixelwise')
S_MODE_TARGETS :  ('Pixelwise',)
RUN_NR :  3
NUM_EPOCHS :  early_stopping
BATCH_SIZE :  8
LEARNING_RATE :  0.005
DEPTH :  3
IN_CHANNELS :  2
CHANNELS_FIRST_CONV :  32
OUT_CHANNELS :  1
FMAPS :  (32, 32, 64, 64)
ACTIVATION :  <class 'torch.nn.modules.activation.ReLU'>
NORMALIZATION :  <class 'torch.nn.modules.batchnorm.BatchNorm3d'>
loss :  MSEL

Check that we can handle the specified target variable and collect the required files.

In [6]:
required_files = []
for i, description in enumerate(dataset_descriptions):
    # print("Target variable:", description["target_variables"])
    if description["target_variables"] == ["dO18"]:
        required_files.append([])
    
        for filename in description["files_used"]:
            if "isotopes" in filename:
                required_files[-1].append(filename)
    else:
        raise NotImplementedError("Currently only d18O is a valid target variable.")

# Load data, especially ground truth data

In [7]:
def get_dataset_dict(path_ds_6_nb, path_ds_5_nb=None):
    """
    Returns a dict that contains the Datasets stored in path_ds_5_nb and path_ds_6_nb. As the names suggest,
    these should be the the datasets containing all points with 5 and all points with six neighbors.
    If only one dataset is used (should be path_ds_6_nb), set the other argument to None.
    """
    import netCDF4
    assert "nbs_6" in path_ds_6_nb
    datasets = {}
    
    datasets["6_nb"] = nc.Dataset(path_ds_6_nb, "a")
    if path_ds_5_nb is not None:
        datasets["5_nb"] = nc.Dataset(path_ds_5_nb, "a")
    return datasets


def get_indices_charts_shape(res):
    """
    Get indices of the two groups: Pixels that have 5 nbs and pixels that have 6 nbs.
    Also return the shape of charts bc we need it later
    """
    ico = Icosahedron(r=res)
    regions, vertices = ico.get_voronoi_regions_vertices()
    charts = ico.get_charts_cut()
    indices_six_nb = []
    indices_five_nb = []
    for i in range(len(regions)):
        if len(regions[i])>5:
            indices_six_nb.append(i)
        else:
            indices_five_nb.append(i)
    # create numpy arrays
    indices_six_nb = np.array(indices_six_nb)
    indices_five_nb = np.array(indices_five_nb)
    return indices_five_nb, indices_six_nb, charts.shape


def combine_datasets(dataset_dict, indices_five_nb, indices_six_nb):
    """
    We need to combine the datasets from the seperate files for points with 5 nbs and points with 6 nbs.
    If there only is a file with six-neighbor points, we fill with zeros. 
    """
    assert "6_nb" in dataset_dict.keys()
    combined_data = np.zeros(dataset_dict["6_nb"].shape[:-1] + (dataset_dict["6_nb"].shape[-1]+10,))
    if "5_nb" in dataset_dict.keys():
        combined_data[:,indices_six_nb] = dataset_dict["6_nb"]
        combined_data[:,indices_five_nb] = dataset_dict["5_nb"]
    else:
        combined_data[:,indices_six_nb] = dataset_dict["6_nb"]
        combined_data[:,indices_five_nb] = 0
    return combined_data

In [8]:
# names of datasets to which we want not to be missing at any timestep.
dnames = [DIRECTORY_DATASETS_ORIGINAL+"xnapa_isotopes.nc",\
          DIRECTORY_DATASETS_ORIGINAL+"xnapa_precip.nc",\
          DIRECTORY_DATASETS_ORIGINAL+"xnapa_slp.nc",\
          DIRECTORY_DATASETS_ORIGINAL+"xnapa_temp.nc"]

def get_shared_timesteps(dataset_names):
    """
    Not all datasets share the same timesteps. The biggest problems occur in the slp dataset. We want to exclude all
    time steps where one of the variables is missing
    """
    
    # get indices of elements that are shared for all variables.
    from functools import reduce
    ts = tuple([nc.Dataset(dataset_name,"a").variables["t"][:].data for dataset_name in dataset_names])
    common_dates = reduce(np.intersect1d, ts)
    
    return common_dates

In [9]:
isotopes_all_datasets = []
for i in range(len(dataset_descriptions)):
    isotopes_all_datasets.append(get_dataset_dict(*required_files[i]))

# somehow two frames are excluded - why???? -> the dO18 datset is 2 timesteps smaller, but the time
# axis actually has the same size...

d18O_ico = []

t = []
t_bnds = []


for i, iso_single_dataset in enumerate(isotopes_all_datasets):
    t.append(iso_single_dataset["6_nb"].variables["t"][:].data)
    t_bnds.append(iso_single_dataset["6_nb"].variables["t_bnds"][:])
    d18O_ico.append({})
    for name, dset in iso_single_dataset.items():  
        d18O_ico[-1][name] = np.squeeze(dset.variables["dO18"][:])

for i in range(len(d18O_ico)):
    for name, array in d18O_ico[i].items():  
        c_dates = get_shared_timesteps(dnames)
        # get the corresponding indices:
        indices = []
        for j, t_ in enumerate(isotopes_all_datasets[i]["6_nb"].variables["t"][:].data):
            if t_ in c_dates:
                indices.append(j)
        indices = np.array(indices)
        index_mask = np.logical_and(isotopes_all_datasets[i]["6_nb"].variables["t"][indices].data // 360 >= 654, \
                                    isotopes_all_datasets[i]["6_nb"].variables["t"][indices].data // 360 < 1654)    
        indices = indices[index_mask]
        
        d18O_ico[i][name] = array[indices,...]

indices_five_nb = []
indices_six_nb = []
cs = []
for description in dataset_descriptions:
    tmp1, tmp2, tmp3 = get_indices_charts_shape(description["RESOLUTION"])
    indices_five_nb.append(tmp1)
    indices_six_nb.append(tmp2)
    cs.append(tmp3)
    
for i in range(len(d18O_ico)):
    d18O_ico[i] = combine_datasets(d18O_ico[i], indices_five_nb[i], indices_six_nb[i])
    d18O_ico[i] = d18O_ico[i].reshape(d18O_ico[i].shape[0],-1,cs[i][-2])

In [10]:
d18O_ico_train = []
t_train = []
t_bnds_train = []

d18O_ico_test = []
t_test = []
t_bnds_test = []

for i, description in enumerate(dataset_descriptions):
    d18O_ico_train.append(d18O_ico[i][description["indices_train"],...])
    t_train.append(t[i][description["indices_train"],...])
    t_bnds_train.append(t_bnds[i][description["indices_train"],...])

    d18O_ico_test.append(d18O_ico[i][description["indices_test"],...])
    t_test.append(t[i][description["indices_test"],...])
    t_bnds_test.append(t_bnds[i][description["indices_test"],...])

In [11]:
rescaled_predictions = []

for i, description in enumerate(model_descriptions):
    rescaled_predictions.append([])
    for j, mode in enumerate(description["S_MODE_TARGETS"]):
        if mode == "Global":
            std = np.mean(np.std(d18O_ico_train[indices_dataset[i]], axis=(0,), keepdims=True), axis=(2,3), keepdims=True)
            std[std==0] = 1
            mean = np.mean(d18O_ico_train[indices_dataset[i]], axis=(0,2,3), keepdims=True)
            rescaled_predictions[-1].append((testset_predictions[i]["predictions"][:,j,...] * std) + mean)             
        elif mode == "Pixelwise":        
            std = np.std(d18O_ico_train[indices_dataset[i]], axis=(0), keepdims=True)
            std[std==0] = 1
            mean = np.mean(d18O_ico_train[indices_dataset[i]], axis=(0), keepdims=True)
            rescaled_predictions[-1].append((testset_predictions[i]["predictions"][:,j,...] * std) + mean)   
        elif mode == "Global_mean_pixelwise_std":
            std = np.mean(np.std(d18O_ico_train[indices_dataset[i]], axis=(0,), keepdims=True), axis=(2,3), keepdims=True)
            std[std==0] = 1
            mean = np.mean(d18O_ico_train[indices_dataset[i]], axis=(0), keepdims=True)
            rescaled_predictions[-1].append((testset_predictions[i]["predictions"][:,j,...] * std) + mean)           
        elif mode == "Pixelwise_mean_global_std":
            std = np.std(d18O_ico_train[indices_dataset[i]], axis=(0), keepdims=True)
            std[std==0] = 1
            mean = np.mean(d18O_ico_train[indices_dataset[i]], axis=(0,2,3), keepdims=True)
            rescaled_predictions[-1].append((testset_predictions[i]["predictions"][:,j,...] * std) + mean)  
        elif mode == "None":
            rescaled_predictions[-1].append(testset_predictions[i]["predictions"])
        else:
            raise NotImplementedError("{} is not a valid keyword for standardization".format(mode))
    rescaled_predictions[-1] = np.stack(rescaled_predictions[-1], axis=1)

In [12]:
def plot_ico_lattice_map(data, vertices, regions, cmap=None, norm=None, show_continents=True, 
                         figsize=(15,10), projection=ccrs.Robinson(), savename=None, title=None, cbar_title=None, title_fontsize=15):
    """
    Plot a single image given in ico_resolution. Assume shape is (npixels,).
    """
    assert len(data.shape) == 1
    if vertices.shape[-1]!=2:  # 'if vertices are not in spherical format already'   
        spherical_vertices = cartesian_to_spherical(vertices)
        spherical_vertices_plot = np.zeros_like(spherical_vertices)
        spherical_vertices_plot[:,0] = spherical_vertices[:,1]# longitude
        spherical_vertices_plot[:,0][spherical_vertices_plot[:,0] == 360] = 0# longitude
        spherical_vertices_plot[:,1] = spherical_vertices[:,0]# latitude
    else:
        spherical_vertices_plot = np.zeros_like(vertices)
        spherical_vertices_plot[:,0] = vertices[:,1]# longitude
        spherical_vertices_plot[:,0][spherical_vertices_plot[:,0] == 360] = 0# longitude
        spherical_vertices_plot[:,1] = vertices[:,0]# latitude        
    import matplotlib.pyplot as plt
    import matplotlib.patches as mpatches
    from matplotlib.collections import PatchCollection

    # plotting
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(projection=projection)

    if cmap is None:
        cmap = plt.get_cmap("plasma")
    if norm is None:
        dmin = min(data)
        dmax = max(data)

    patches = []


    for i in range(len(regions)):
        tmp = spherical_vertices_plot[regions[i]]
        # Polygons that lie close to the 0°-360° continuity get connected wrongly by cartopy. We fix this for now
        # Solution is not perfect.
        if np.amax(tmp[:,0])- np.amin(tmp[:,0]) > 180: 
            tmp[tmp>180] = tmp[tmp>180]-360
        polygon = mpatches.Polygon(tmp,
                          transform=ccrs.PlateCarree())
        if norm is None:
            polygon.set_color(cmap(np.array((data[i]-dmin)/(dmax-dmin))))
        else:
            polygon.set_color(cmap(norm(data[i])))
        ax.add_patch(polygon)
        
    cbar = fig.colorbar(
        matplotlib.cm.ScalarMappable(cmap=cmap, norm=norm),
        # ticks=bounds,
        spacing='proportional',
        orientation='horizontal',
    ) 
    if cbar_title is None:
        cbar.set_label(r"1 - (RMSE/$\sigma_{test})$", fontsize = title_fontsize)
    else:
        cbar.set_label(cbar_title, fontsize = title_fontsize)
    cbar.ax.tick_params(labelsize=20)
    ax.set_global()
    ax.coastlines(alpha=0.5)
    if title is not None:
        ax.set_title(title, fontsize=title_fontsize)
    if savename is not None:
        plt.savefig(DIRECTORY_IMAGES+savename)
    plt.show()
    
def cartesian_to_spherical(data):
    """
    convert cartesian coordinates to spherical coordinates
    Use answer to:
    https://stackoverflow.com/questions/4116658/faster-numpy-cartesian-to-spherical-coordinate-conversion
    """
    # takes list xyz (single coord)
    x = data[..., 0]
    y = data[..., 1]
    z = data[..., 2]
    r = np.sqrt(x * x + y * y + z * z)
    # format in HadCM3: lat:(-90,90), lon(0,360)
    theta = 90 - np.arccos(z / r) * 180 / np.pi  # to degrees
    phi = 180 + np.arctan2(y, x) * 180 / np.pi
    return np.array([theta, phi]).transpose((1, 0))  # careful, this will only work if the shape is correct

In [13]:
# calibrate colorscale such that all simulations share the same min and max
vmin = min([np.amin(r_pred[0]) for r_pred in rescaled_predictions] + [np.amin(gt[0]) for gt in d18O_ico_test])
vmax = max([np.amax(r_pred[0]) for r_pred in rescaled_predictions] + [np.amax(gt[0]) for gt in d18O_ico_test])

norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax, clip=False)

In [14]:
ico = []
regions = []
vertices = []
for description in dataset_descriptions:
    ico.append(Icosahedron(r=description["RESOLUTION"]))
    tmp1, tmp2 = ico[-1].get_voronoi_regions_vertices()
    regions.append(tmp1)
    vertices.append(tmp2)

In [16]:
"""
for i, i_j in enumerate(indices_dataset):
    # for key, value in dataset_descriptions[i_j].items():
        # print(key,": ", value)
    for key, value in model_descriptions[i].items():
        print(key,": ", value)
    plot_ico_lattice_map(rescaled_predictions[i][0,0].flatten(), vertices[i_j], regions[i_j], norm=norm, cbar_title=r"{}^{18}\delta(O)}")
"""    
print("uncomment this to get rescaled sample predictions.")

uncomment this to get rescaled sample predictions.


In [17]:
def RMSE(predictions, targets):
    """
    needs predictions in shape (n_predictions, lat*lon).
    """
    from sklearn.metrics import mean_squared_error
    rmse = np.zeros((predictions.shape[1]))
    for i in range(predictions.shape[1]):
        rmse[i] = mean_squared_error(targets[:,i], predictions[:,i], squared=False)
    return rmse

In [18]:
rmse_ico = []

for i, rescaled_prediction in enumerate(rescaled_predictions):
    rmse_ico.append(RMSE(rescaled_prediction.reshape(rescaled_prediction.shape[0],-1), d18O_ico_test[indices_dataset[i]].reshape(d18O_ico_test[indices_dataset[i]].shape[0],-1)))

# compute explained variance by dividing rmse by standarddeviation on test set.
metric_ico = []
for i, rmse in enumerate(rmse_ico):
    metric_ico.append(1 - rmse/np.std(d18O_ico_test[indices_dataset[i]].reshape(d18O_ico_test[indices_dataset[i]].shape[0],-1), axis=0))

In [27]:
#set up colormap
colors = ["#67001f","#b2182b","#d6604d","#f4a582","#fddbc7","#d1e5f0","#92c5de","#4393c3","#2166ac","#053061"]
lim = 1 # math.ceil(np.max(metric))
bounds = np.concatenate((np.array([-1.0,-0.8,-0.6,-0.4,-0.2]),np.linspace(0, lim, 6)))
cmap= matplotlib.colors.ListedColormap(colors)
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)

In [20]:
"""
for i, data in enumerate(metric_ico):
    print("Predictor_vars: ", dataset_descriptions[indices_dataset[i]]["predictor_variables"])
    for key, value in model_descriptions[i].items():
        print(key,": ", value)
    plot_ico_lattice_map(data, vertices[indices_dataset[i]], regions[indices_dataset[i]], cmap=cmap, norm=norm, 
                         title=r"$R^2$ score, " + "Global area weighted average: {:.3f}".format(np.average(data)))
"""
print("uncomment this to get performance maps for individual runs.")

uncomment this to get performance maps for individual runs.
